In [1]:
!pwd

/bin/bash: /home/hayade/software/anaconda3/envs/myenv/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/hayade/project/mmtracking/notebooks


In [2]:
import os
import os.path as osp

import mmcv
import tempfile
from collections import defaultdict
from mmtrack.apis import inference_mot, init_model

/home/hayade/software/anaconda3/envs/myenv/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
input_folder = "../data/DNP/video/"
output_ano = "../output/DNP/anotations/"
output_vid = "../output/DNP/videos/"

In [10]:
mot_config = '../configs/mot/bytetrack/bytetrack_yolox_x_crowdhuman_mot17-private-half.py'
mot_checkpoint = '../checkpoints/bytetrack/bytetrack_yolox_x_crowdhuman_mot17-private-half_20211218_205500-1985c9f0.pth'
mot_model = init_model(mot_config, mot_checkpoint, device='cuda:0')

2023-06-03 09:30:41,011 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-06-03 09:30:41,060 - mmcv - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-06-03 09:30:41,061 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-06-03 09:30:41,062 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-06-03 09:30:41,532 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 1])

load checkpoint from local path: ../checkpoints/bytetrack/bytetrack_yolox_x_crowdhuman_mot17-private-half_20211218_205500-1985c9f0.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_backbone_stage1_1_main_conv_bn_running_mean, em

In [11]:
mot_model.CLASSES

('pedestrian',)

In [5]:
for input_file in os.listdir(input_folder):
    print(f"==========={input_file}==========")

    input_video = osp.join(input_folder, input_file)
    input_file = input_file.split(".")[0]
    
    imgs = mmcv.VideoReader(input_video)

    # build the model from a config file
    
    prog_bar = mmcv.ProgressBar(len(imgs))
    out_dir = tempfile.TemporaryDirectory()
    out_path = out_dir.name

    pred_file = osp.join(output_ano, input_file + ".json")
    output = osp.join(output_vid, input_file + ".mp4")
    if (osp.exists(pred_file) and osp.exists(output)):
          print()
          continue

    out_data = defaultdict(list)

    # test and show/save the images
    for i, img in enumerate(imgs):
            result = inference_mot(mot_model, img, frame_id=i)
            out_data[i].append(result)
            mot_model.show_result(
                    img,
                    result,
                    show=False,
                    wait_time=int(1000. / imgs.fps),
                    out_file=f'{out_path}/{i:06d}.jpg')
            prog_bar.update()

    # print out pred in json format
    mmcv.dump(out_data, pred_file)

    
    print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
    mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
    out_dir.cleanup()
    
    print()

===========output015.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output017.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output001.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output010.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output024.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output022.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output013.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output006.mp4==========
[                                                  ] 0/1500, elapsed: 0s, ETA:
===========output005.mp4==========
[                                                  ] 

/home/hayade/software/anaconda3/envs/myenv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 3.4 task/s, elapsed: 435s, ETA:     0s
 making the output video at ../output/DNP/videos/output018.mp4 with a FPS of 25.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 27.2 task/s, elapsed: 55s, ETA:     0s

===========output028.mp4==========
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 3.4 task/s, elapsed: 442s, ETA:     0s
 making the output video at ../output/DNP/videos/output028.mp4 with a FPS of 25.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 27.6 task/s, elapsed: 54s, ETA:     0s

===========output027.mp4==========
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 3.5 task/s, elapsed: 432s, ETA:     0s
 making the output video at ../output/DNP/videos/output027.mp4 with a FPS of 25.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 28.4 task/s, elapsed: 53s, ETA:     0s

===========output009.mp4==========
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1500/1500, 3.5 task/s, elapsed: 429s, ETA:     0s
 making the output video at ../output/DNP/videos/output009.mp4 with a FPS of 25.0
[>>>